### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
school_data_complete

Student ID       student_name gender grade         school_name  \
0               0       Paul Bradley      M   9th   Huang High School   
1               1       Victor Smith      M  12th   Huang High School   
2               2    Kevin Rodriguez      M  12th   Huang High School   
3               3  Dr. Richard Scott      M  12th   Huang High School   
4               4         Bonnie Ray      F   9th   Huang High School   
...           ...                ...    ...   ...                 ...   
39165       39165       Donna Howard      F  12th  Thomas High School   
39166       39166          Dawn Bell      F  10th  Thomas High School   
39167       39167     Rebecca Tanner      F   9th  Thomas High School   
39168       39168       Desiree Kidd      F  10th  Thomas High School   
39169       39169    Carolyn Jackson      F  11th  Thomas High School   

       reading_score  math_score  School ID      type  size   budget  
0                 66          79          0  District  2917  1910635  
1                 94          61          0  District  2917  1910635  
2                 90          60          0  District  2917  1910635  
3                 67          58          0  District  2917  1910635  
4                 97          84          0  District  2917  1910635  
...              ...         ...        ...       ...   ...      ...  
39165             99          90         14   Charter  1635  1043130  
39166             95          70         14   Charter  1635  1043130  
39167             73          84         14   Charter  1635  1043130  
39168             99          90         14   Charter  1635  1043130  
39169             95          75         14   Charter  1635  1043130  

[39170 rows x 11 columns]

In [3]:
schools_df = school_data_complete["school_name"].nunique()
student_count = school_data_complete["student_name"].count()
total_budget = pd.unique(school_data_complete["budget"]).sum()
average_math = school_data_complete["math_score"].mean()
average_reading = school_data_complete["reading_score"].mean()
math_pass = school_data_complete.loc[school_data_complete["math_score"] > 70,:]
math_pass_rate = (math_pass["math_score"].count() / 
                  school_data_complete["math_score"].count())*100
reading_pass = school_data_complete.loc[school_data_complete["reading_score"] > 70,:]
reading_pass_rate = (reading_pass["reading_score"].count() / 
                     school_data_complete["reading_score"].count())*100
combined_pass = school_data_complete.loc[(school_data_complete["math_score"] > 70) &
                                         (school_data_complete["reading_score"] > 70),:]
combined_pass_rate = (combined_pass["math_score"].count() / 
                      school_data_complete["math_score"].count())*100



district_summary_df = pd.DataFrame({"Total Schools": [schools_df],
                                    "Total Students": [student_count],
                                    "Total Budget (USD)": [total_budget],
                                    "Average Math Score": [average_math],
                                    "Average Reading Score": [average_reading],
                                    "Math Pass Rate (%)": [math_pass_rate],
                                    "Reading Pass Rate (%)":[reading_pass_rate],
                                    "Math And Reading Pass Rate (%)":[combined_pass_rate]
                                   })


district_summary_df["Total Students"] = district_summary_df["Total Students"].astype(float).map("{:,.0f}".format)
district_summary_df["Total Budget (USD)"] = district_summary_df["Total Budget (USD)"].astype(float).map("${:,.2f}".format)
district_summary_df["Math Pass Rate (%)"] = district_summary_df["Math Pass Rate (%)"].astype(float).map("{:.2f}".format)
district_summary_df["Reading Pass Rate (%)"] = district_summary_df["Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
district_summary_df["Math And Reading Pass Rate (%)"] = district_summary_df["Math And Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
district_summary_df["Average Math Score"] = district_summary_df["Average Math Score"].astype(float).map("{:.2f}".format)
district_summary_df["Average Reading Score"] = district_summary_df["Average Reading Score"].astype(float).map("{:.2f}".format)



district_summary_df

Total Schools Total Students Total Budget (USD) Average Math Score  \
0             15         39,170     $24,649,428.00              78.99   

  Average Reading Score Math Pass Rate (%) Reading Pass Rate (%)  \
0                 81.88              72.39                 82.97   

  Math And Reading Pass Rate (%)  
0                          60.80

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [4]:
schools_sorted = school_data_complete.sort_values(by=["school_name"])
schools_columns = schools_sorted[["school_name","type","budget","size","School ID"]]
organized_schools_sorted = schools_columns.rename(columns={"school_name":"School Name",
                                          "type":"School Type",
                                          "budget":"School Budget", 
                                          "size":"Number of Students"})

unique_schools = organized_schools_sorted.drop_duplicates(subset = ["School Name"])
unique_schools["Budget per Student"] = unique_schools["School Budget"]/unique_schools["Number of Students"]

test_scores = school_data_complete.groupby(["school_name"]).mean()
test_scores = test_scores[["math_score","reading_score","School ID"]]
test_scores_df = pd.merge(unique_schools, test_scores, how="left", on=["School ID", "School ID"])

math_passing = school_data_complete[school_data_complete['math_score'] >= 70].groupby(['school_name'])
reading_passing = school_data_complete[school_data_complete['reading_score'] >= 70].groupby(['school_name'])
total_passing = school_data_complete[(school_data_complete['reading_score'] >= 70) &
                                    (school_data_complete['math_score'] >= 70)].groupby(['school_name'])

passing_percentage = pd.DataFrame({"Math Pass Rate (%)": math_passing["school_name"].count(),
                   "Reading Pass Rate (%)": reading_passing["school_name"].count(),
                   "Math and Reading Pass Rate (%)": total_passing["school_name"].count()
                  })
passing_percentage = passing_percentage.reset_index()

school_summary = pd.merge(test_scores_df, passing_percentage, how="left", left_index = True, right_index = True)
school_summary["Math Pass Rate (%)"] = (school_summary["Math Pass Rate (%)"]/school_summary["Number of Students"])*100
school_summary["Reading Pass Rate (%)"] = (school_summary["Reading Pass Rate (%)"]/school_summary["Number of Students"])*100
school_summary["Math and Reading Pass Rate (%)"] = (school_summary["Math and Reading Pass Rate (%)"]/school_summary["Number of Students"])*100

school_summary = school_summary.drop(columns = ["school_name"])
school_summary = school_summary.drop(columns = ["School ID"])

school_metrics = school_summary.set_index(["School Name"])


school_metrics = school_metrics.rename(columns={"School Budget":"School Budget (USD)",
                                                "Budget per Student":"Budget Per Student (USD)",
                                                "math_score":"Average Math Score",
                                                "reading_score":"Average Reading Score"})

school_metrics["School Budget (USD)"] = school_metrics["School Budget (USD)"].astype(float).map("${:,.2f}".format)
school_metrics["Budget Per Student (USD)"] = school_metrics["Budget Per Student (USD)"].astype(float).map("${:,.2f}".format)
school_metrics["Math Pass Rate (%)"] = school_metrics["Math Pass Rate (%)"].astype(float).map("{:.2f}".format)
school_metrics["Reading Pass Rate (%)"] = school_metrics["Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
school_metrics["Math and Reading Pass Rate (%)"] = school_metrics["Math and Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
school_metrics["Average Math Score"] = school_metrics["Average Math Score"].astype(float).map("{:.2f}".format)
school_metrics["Average Reading Score"] = school_metrics["Average Reading Score"].astype(float).map("{:.2f}".format)


school_metrics

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


School Type School Budget (USD)  Number of Students  \
School Name                                                                 
Bailey High School       District       $3,124,928.00                4976   
Cabrera High School       Charter       $1,081,356.00                1858   
Figueroa High School     District       $1,884,411.00                2949   
Ford High School         District       $1,763,916.00                2739   
Griffin High School       Charter         $917,500.00                1468   
Hernandez High School    District       $3,022,020.00                4635   
Holden High School        Charter         $248,087.00                 427   
Huang High School        District       $1,910,635.00                2917   
Johnson High School      District       $3,094,650.00                4761   
Pena High School          Charter         $585,858.00                 962   
Rodriguez High School    District       $2,547,363.00                3999   
Shelton High School       Charter       $1,056,600.00                1761   
Thomas High School        Charter       $1,043,130.00                1635   
Wilson High School        Charter       $1,319,574.00                2283   
Wright High School        Charter       $1,049,400.00                1800   

                      Budget Per Student (USD) Average Math Score  \
School Name                                                         
Bailey High School                     $628.00              77.05   
Cabrera High School                    $582.00              83.06   
Figueroa High School                   $639.00              76.71   
Ford High School                       $644.00              77.10   
Griffin High School                    $625.00              83.35   
Hernandez High School                  $652.00              77.29   
Holden High School                     $581.00              83.80   
Huang High School                      $655.00              76.63   
Johnson High School                    $650.00              77.07   
Pena High School                       $609.00              83.84   
Rodriguez High School                  $637.00              76.84   
Shelton High School                    $600.00              83.36   
Thomas High School                     $638.00              83.42   
Wilson High School                     $578.00              83.27   
Wright High School                     $583.00              83.68   

                      Average Reading Score Math Pass Rate (%)  \
School Name                                                      
Bailey High School                    81.03              66.68   
Cabrera High School                   83.98              94.13   
Figueroa High School                  81.16              65.99   
Ford High School                      80.75              68.31   
Griffin High School                   83.82              93.39   
Hernandez High School                 80.93              66.75   
Holden High School                    83.81              92.51   
Huang High School                     81.18              65.68   
Johnson High School                   80.97              66.06   
Pena High School                      84.04              94.59   
Rodriguez High School                 80.74              66.37   
Shelton High School                   83.73              93.87   
Thomas High School                    83.85              93.27   
Wilson High School                    83.99              93.87   
Wright High School                    83.95              93.33   

                      Reading Pass Rate (%) Math and Reading Pass Rate (%)  
School Name                                                                 
Bailey High School                    81.93                          54.64  
Cabrera High School                   97.04                          91.33  
Figueroa High School                  80.74                          53.20  
Ford High School              

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [5]:
top_performing_overall = school_summary.sort_values("Math and Reading Pass Rate (%)", ascending = False)
top_performing_overall.head(5)

School Name School Type  School Budget  Number of Students  \
1   Cabrera High School     Charter        1081356                1858   
12   Thomas High School     Charter        1043130                1635   
4   Griffin High School     Charter         917500                1468   
13   Wilson High School     Charter        1319574                2283   
9      Pena High School     Charter         585858                 962   

    Budget per Student  math_score  reading_score  Math Pass Rate (%)  \
1                582.0   83.061895      83.975780           94.133477   
12               638.0   83.418349      83.848930           93.272171   
4                625.0   83.351499      83.816757           93.392371   
13               578.0   83.274201      83.989488           93.867718   
9                609.0   83.839917      84.044699           94.594595   

    Reading Pass Rate (%)  Math and Reading Pass Rate (%)  
1               97.039828                       91.334769  
12              97.308869                       90.948012  
4               97.138965                       90.599455  
13              96.539641                       90.582567  
9               95.945946                       90.540541

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [6]:
lowest_performing_overall = school_summary.sort_values("Math and Reading Pass Rate (%)", ascending = True)
lowest_performing_overall.head(5)

School Name School Type  School Budget  Number of Students  \
10  Rodriguez High School    District        2547363                3999   
2    Figueroa High School    District        1884411                2949   
7       Huang High School    District        1910635                2917   
5   Hernandez High School    District        3022020                4635   
8     Johnson High School    District        3094650                4761   

    Budget per Student  math_score  reading_score  Math Pass Rate (%)  \
10               637.0   76.842711      80.744686           66.366592   
2                639.0   76.711767      81.158020           65.988471   
7                655.0   76.629414      81.182722           65.683922   
5                652.0   77.289752      80.934412           66.752967   
8                650.0   77.072464      80.966394           66.057551   

    Reading Pass Rate (%)  Math and Reading Pass Rate (%)  
10              80.220055                       52.988247  
2               80.739234                       53.204476  
7               81.316421                       53.513884  
5               80.862999                       53.527508  
8               81.222432                       53.539172

## Math Scores by Grade

* Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
ninth_grade = school_data_complete.loc[school_data_complete["grade"] == "9th" ,:]
tenth_grade = school_data_complete.loc[school_data_complete["grade"] == "10th" ,:]
eleventh_grade = school_data_complete.loc[school_data_complete["grade"] == "11th" ,:]
twelth_grade = school_data_complete.loc[school_data_complete["grade"] == "12th" ,:]

school_names = school_data_complete["school_name"].unique()

ninth_math_average = ninth_grade["math_score"].groupby(ninth_grade["school_name"]).mean()
tenth_math_average = tenth_grade["math_score"].groupby(tenth_grade["school_name"]).mean()
eleventh_math_average = eleventh_grade["math_score"].groupby(eleventh_grade["school_name"]).mean()
twelth_math_average = twelth_grade["math_score"].groupby(twelth_grade["school_name"]).mean()

average_math_per_grade = pd.DataFrame({"9th Grade Math Average": ninth_math_average,
                                    "10th Grade Math Average": tenth_math_average,
                                    "11th Grade Math Average": eleventh_math_average,
                                    "12th Grade Math Average": twelth_math_average,
                                   })

average_math_per_grade["9th Grade Math Average"] = average_math_per_grade["9th Grade Math Average"].astype(float).map("{:.2f}".format)
average_math_per_grade["10th Grade Math Average"] = average_math_per_grade["10th Grade Math Average"].astype(float).map("{:.2f}".format)
average_math_per_grade["11th Grade Math Average"] = average_math_per_grade["11th Grade Math Average"].astype(float).map("{:.2f}".format)
average_math_per_grade["12th Grade Math Average"] = average_math_per_grade["12th Grade Math Average"].astype(float).map("{:.2f}".format)

average_math_per_grade

9th Grade Math Average 10th Grade Math Average  \
school_name                                                            
Bailey High School                     77.08                   77.00   
Cabrera High School                    83.09                   83.15   
Figueroa High School                   76.40                   76.54   
Ford High School                       77.36                   77.67   
Griffin High School                    82.04                   84.23   
Hernandez High School                  77.44                   77.34   
Holden High School                     83.79                   83.43   
Huang High School                      77.03                   75.91   
Johnson High School                    77.19                   76.69   
Pena High School                       83.63                   83.37   
Rodriguez High School                  76.86                   76.61   
Shelton High School                    83.42                   82.92   
Thomas High School                     83.59                   83.09   
Wilson High School                     83.09                   83.72   
Wright High School                     83.26                   84.01   

                      11th Grade Math Average 12th Grade Math Average  
school_name                                                            
Bailey High School                      77.52                   76.49  
Cabrera High School                     82.77                   83.28  
Figueroa High School                    76.88                   77.15  
Ford High School                        76.92                   76.18  
Griffin High School                     83.84                   83.36  
Hernandez High School                   77.14                   77.19  
Holden High School                      85.00                   82.86  
Huang High School                       76.45                   77.23  
Johnson High School                     77.49                   76.86  
Pena High School                        84.33                   84.12  
Rodriguez High School                   76.40                   77.69  
Shelton High School                     83.38                   83.78  
Thomas High School                      83.50                   83.50  
Wilson High School                      83.20                   83.04  
Wright High School                      83.84                   83.64

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
ninth_reading_average = ninth_grade["reading_score"].groupby(ninth_grade["school_name"]).mean()
tenth_reading_average = tenth_grade["reading_score"].groupby(tenth_grade["school_name"]).mean()
eleventh_reading_average = eleventh_grade["reading_score"].groupby(eleventh_grade["school_name"]).mean()
twelth_reading_average = twelth_grade["reading_score"].groupby(twelth_grade["school_name"]).mean()

average_reading_per_grade = pd.DataFrame({"9th Grade Reading Average": ninth_reading_average,
                                    "10th Grade Reading Average": tenth_reading_average,
                                    "11th Grade Reading Average": eleventh_reading_average,
                                    "12th Grade Reading Average": twelth_reading_average,
                                   })

average_reading_per_grade["9th Grade Reading Average"] = average_reading_per_grade["9th Grade Reading Average"].astype(float).map("{:.2f}".format)
average_reading_per_grade["10th Grade Reading Average"] = average_reading_per_grade["10th Grade Reading Average"].astype(float).map("{:.2f}".format)
average_reading_per_grade["11th Grade Reading Average"] = average_reading_per_grade["11th Grade Reading Average"].astype(float).map("{:.2f}".format)
average_reading_per_grade["12th Grade Reading Average"] = average_reading_per_grade["12th Grade Reading Average"].astype(float).map("{:.2f}".format)


average_reading_per_grade

9th Grade Reading Average 10th Grade Reading Average  \
school_name                                                                  
Bailey High School                        81.30                      80.91   
Cabrera High School                       83.68                      84.25   
Figueroa High School                      81.20                      81.41   
Ford High School                          80.63                      81.26   
Griffin High School                       83.37                      83.71   
Hernandez High School                     80.87                      80.66   
Holden High School                        83.68                      83.32   
Huang High School                         81.29                      81.51   
Johnson High School                       81.26                      80.77   
Pena High School                          83.81                      83.61   
Rodriguez High School                     80.99                      80.63   
Shelton High School                       84.12                      83.44   
Thomas High School                        83.73                      84.25   
Wilson High School                        83.94                      84.02   
Wright High School                        83.83                      83.81   

                      11th Grade Reading Average 12th Grade Reading Average  
school_name                                                                  
Bailey High School                         80.95                      80.91  
Cabrera High School                        83.79                      84.29  
Figueroa High School                       80.64                      81.38  
Ford High School                           80.40                      80.66  
Griffin High School                        84.29                      84.01  
Hernandez High School                      81.40                      80.86  
Holden High School                         83.82                      84.70  
Huang High School                          81.42                      80.31  
Johnson High School                        80.62                      81.23  
Pena High School                           84.34                      84.59  
Rodriguez High School                      80.86                      80.38  
Shelton High School                        84.37                      82.78  
Thomas High School                         83.59                      83.83  
Wilson High School                         83.76                      84.32  
Wright High School                         84.16                      84.07

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
spending_bins = [0, 585, 610, 635, 660]
spending_ranges = ["<584", "585-609", "610-634", "635-660"]

spending_summary = school_summary[["School Name",
                                    "math_score",
                                    "reading_score",
                                    "Math Pass Rate (%)",
                                    "Reading Pass Rate (%)",
                                    "Math and Reading Pass Rate (%)"]]

spending_summary["Spending Range (USD per student)"] = pd.cut(school_summary["Budget per Student"], spending_bins, labels=spending_ranges, include_lowest=True)
spending_summary = spending_summary.groupby(["Spending Range (USD per student)"]).mean()

spending_summary = spending_summary.rename(columns={"math_score":"Average Math Score",
                                                "reading_score":"Average Reading Score"})

spending_summary["Math Pass Rate (%)"] = spending_summary["Math Pass Rate (%)"].astype(float).map("{:.2f}".format)
spending_summary["Reading Pass Rate (%)"] = spending_summary["Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
spending_summary["Math and Reading Pass Rate (%)"] = spending_summary["Math and Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
spending_summary["Average Math Score"] = spending_summary["Average Math Score"].astype(float).map("{:.2f}".format)
spending_summary["Average Reading Score"] = spending_summary["Average Reading Score"].astype(float).map("{:.2f}".format)


spending_summary

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Average Math Score Average Reading Score  \
Spending Range (USD per student)                                            
<584                                          83.46                 83.93   
585-609                                       83.60                 83.89   
610-634                                       80.20                 82.43   
635-660                                       77.87                 81.37   

                                 Math Pass Rate (%) Reading Pass Rate (%)  \
Spending Range (USD per student)                                            
<584                                          93.46                 96.61   
585-609                                       94.23                 95.90   
610-634                                       80.04                 89.54   
635-660                                       70.35                 83.00   

                                 Math and Reading Pass Rate (%)  
Spending Range (USD per student)                                 
<584                                                      90.37  
585-609                                                   90.22  
610-634                                                   72.62  
635-660                                                   58.86

## Scores by School Size

* Perform the same operations as above, based on school size.

In [10]:
school_size_bins = [0, 1500, 3000, 4500, 6000]
student_body = ["<1500", "1501-3000", "3001-4500", "4501-6000"]

school_size_summary = school_summary[["School Name",
                                    "math_score",
                                    "reading_score",
                                    "Math Pass Rate (%)",
                                    "Reading Pass Rate (%)",
                                    "Math and Reading Pass Rate (%)"]]

school_size_summary["School Body Size"] = pd.cut(school_summary["Number of Students"], school_size_bins, labels=student_body, include_lowest=True)
school_size_summary = school_size_summary.groupby(["School Body Size"]).mean()

school_size_summary = school_size_summary.rename(columns={"math_score":"Average Math Score",
                                                "reading_score":"Average Reading Score"})

school_size_summary["Math Pass Rate (%)"] = school_size_summary["Math Pass Rate (%)"].astype(float).map("{:.2f}".format)
school_size_summary["Reading Pass Rate (%)"] = school_size_summary["Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
school_size_summary["Math and Reading Pass Rate (%)"] = school_size_summary["Math and Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
school_size_summary["Average Math Score"] = school_size_summary["Average Math Score"].astype(float).map("{:.2f}".format)
school_size_summary["Average Reading Score"] = school_size_summary["Average Reading Score"].astype(float).map("{:.2f}".format)


school_size_summary

/Applications/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Average Math Score Average Reading Score Math Pass Rate (%)  \
School Body Size                                                               
<1500                         83.66                 83.89              93.50   
1501-3000                     80.90                 82.82              83.56   
3001-4500                     76.84                 80.74              66.37   
4501-6000                     77.14                 80.98              66.50   

                 Reading Pass Rate (%) Math and Reading Pass Rate (%)  
School Body Size                                                       
<1500                            96.45                          90.12  
1501-3000                        90.59                          76.76  
3001-4500                        80.22                          52.99  
4501-6000                        81.34                          53.90

## Scores by School Type

* Perform the same operations as above, based on school type

In [11]:
school_type_summary = school_summary.groupby(["School Type"]).mean()

school_type_summary = school_type_summary[["math_score",
                                    "reading_score",
                                    "Math Pass Rate (%)",
                                    "Reading Pass Rate (%)",
                                    "Math and Reading Pass Rate (%)"]]

school_type_summary = school_type_summary.rename(columns={"math_score":"Average Math Score",
                                                "reading_score":"Average Reading Score"})

school_type_summary["Math Pass Rate (%)"] = school_type_summary["Math Pass Rate (%)"].astype(float).map("{:.2f}".format)
school_type_summary["Reading Pass Rate (%)"] = school_type_summary["Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
school_type_summary["Math and Reading Pass Rate (%)"] = school_type_summary["Math and Reading Pass Rate (%)"].astype(float).map("{:.2f}".format)
school_type_summary["Average Math Score"] = school_type_summary["Average Math Score"].astype(float).map("{:.2f}".format)
school_type_summary["Average Reading Score"] = school_type_summary["Average Reading Score"].astype(float).map("{:.2f}".format)


school_type_summary

Average Math Score Average Reading Score Math Pass Rate (%)  \
School Type                                                               
Charter                  83.47                 83.90              93.62   
District                 76.96                 80.97              66.55   

            Reading Pass Rate (%) Math and Reading Pass Rate (%)  
School Type                                                       
Charter                     96.59                          90.43  
District                    80.80                          53.67